In [55]:
import pandas as pd

# read in History
hist_df = pd.read_csv("datasets/chrome_hist.csv")

hist_df.head()

,id,url,title,visit_count,typed_count,last_visit_time,hidden
0,348,https://www.facebook.com/profile.php?id=100004...,Rohan Mishra,268,129,13162785960093394,0
1,551,https://www.messenger.com/t/1002100799843065,Messenger,115,0,13162868680446667,0
2,704,https://github.com/Meeshbhoombah,Meeshbhoombah (Rohan Mishra),200,53,13162871708126532,0
3,1110,https://www.makeschool.com/users/auth/facebook,Log into Facebook | Facebook,5,0,13157852987745316,0
4,1347,https://www.messenger.com/t/dhruv.mand,(1) Messenger,7,0,13160376562930578,0


## Obeservations
Looking at the dataset, each row seems to be a unique url. It seems to me that a URL can only appear once in the database, therefore, each URL contains the visit count so that the total number of visits to a website can be determined while still avoiding duplication.

This data will yield variable results if parsed through incorrectly because of the value of certain attributes, like the `title` attribute, do not follow a set pattern. This is **bad news bears**. Thankfully, the website can be guessed from the `url_name` by splitting the string after `https://www.`, which is a prefix that they each shares. The end of a websites's name is delimited by `.com`.

## Time Heatmap
I want to gain an insight into how often I am browsing the internet on my Mac so I decided to create a time heatmap of my browsing data so I can easily interpret that. I want to see an hourly breakdown of my browsing activity on a daily basis, so I need to convert the `last_visit_time` attribute to a readable date format. 

- Inital unable to read error
- Googling to find format
- Aftewards integer size error
- Fix with normalization

In [83]:
import datetime

last_visit_times = hist_df["last_visit_time"].values

# splice for testing
last_visit_times = last_visit_times[:5]

# convert times
for visit_time in last_visit_times:
    
    # convert numpy.int64 to int
    visit_time = np.asscalar(visit_time)
    
    # Chrome's timestamp is formatted as 
    # microseconds from January 1, 1601
    epoch_start = datetime.datetime(1601, 1, 1)
    
    # delta
    delta = datetime.timedelta(microseconds = int(visit_time))
    
    # comebine
    visit_time = epoch_start + delta
    
    # isoformat
    visit_time = visit_time.isoformat()
    
    print(visit_time)

2018-02-11T01:26:00.093394
2018-02-12T00:24:40.446667
2018-02-12T01:15:08.126532
2017-12-15T23:09:47.745316
2018-01-14T04:09:22.930578


## Wakatime
We can fill in the gaps in my programming history by using 

In [ ]:
import hashlib
import os
import sys
from rauth import OAuth2Service

if sys.version_info[0] == 3:
    raw_input = input

client_id = raw_input('Enter your App Id: ')
secret = raw_input('Enter your App Secret: ')

service = OAuth2Service(
    client_id=client_id,  # App ID from https://wakatime.com/apps
    client_secret=secret,  # App Secret from https://wakatime.com/apps
    name='wakatime',
    authorize_url='https://wakatime.com/oauth/authorize',
    access_token_url='https://wakatime.com/oauth/token',
    base_url='https://wakatime.com/api/v1/')

redirect_uri = 'https://wakatime.com/oauth/test'
state = hashlib.sha1(os.urandom(40)).hexdigest()
params = {'scope': 'email,read_stats',
          'response_type': 'code',
          'state': state,
          'redirect_uri': redirect_uri}

url = service.get_authorize_url(**params)

print('**** Visit this url in your browser ****'.format(url=url))
print('*' * 80)
print(url)
print('*' * 80)
print('**** After clicking Authorize, paste code here and press Enter ****')
code = raw_input('Enter code from url: ')

# Make sure returned state has not changed for security reasons, and exchange
# code for an Access Token.
headers = {'Accept': 'application/x-www-form-urlencoded'}
print('Getting an access token...')
session = service.get_auth_session(headers=headers,
                                   data={'code': code,
                                         'grant_type': 'authorization_code',
                                         'redirect_uri': redirect_uri})

print('Getting current user from API...')
user = session.get('users/current').json()
print('Authenticated via OAuth as {0}'.format(user['data']['email']))
print("Getting user's coding stats from API...")
stats = session.get('users/current/stats')
print(stats.text)